In [2]:
from snapi_py_client.snapi_bridge import StocknoteAPIPythonBridge
import pandas as pd
import config
import requests
import json
import time
from datetime import datetime, timedelta

ImportError: cannot import name 'WebSocketApp' from 'websocket' (/home/rishi/Open-TA/fast-venv/lib/python3.10/site-packages/websocket/__init__.py)

In [4]:
from snapi_py_client.snapi_bridge import StocknoteAPIPythonBridge

ImportError: cannot import name 'WebSocketApp' from 'websocket' (/home/rishi/Open-TA/fast-venv/lib/python3.10/site-packages/websocket/__init__.py)

In [2]:
samco=StocknoteAPIPythonBridge()
login=samco.login(body={"userId":config.username,'password':'samco@12345','yob':config.yob})
login = json.loads(login)
login

{'serverTime': '26/12/23 18:06:23',
 'msgId': '64aacbcd-dac5-4c4a-92ff-f06835770ff8',
 'status': 'Success',
 'statusMessage': 'Login session token generated successfully ',
 'sessionToken': '9d480496f23108cdb7db0124e651b553',
 'accountID': 'DR46682',
 'accountName': 'RISHI GUPTA',
 'exchangeList': ['NFO', 'NSE', 'MCX', 'CDS', 'BFO'],
 'orderTypeList': ['MKT', 'L', 'SL'],
 'productList': ['MIS', 'CNC', 'NRML', 'CO']}

In [3]:
samco.set_session_token(sessionToken=login['sessionToken'])
print("Login details",login)

Login details {'serverTime': '26/12/23 18:06:23', 'msgId': '64aacbcd-dac5-4c4a-92ff-f06835770ff8', 'status': 'Success', 'statusMessage': 'Login session token generated successfully ', 'sessionToken': '9d480496f23108cdb7db0124e651b553', 'accountID': 'DR46682', 'accountName': 'RISHI GUPTA', 'exchangeList': ['NFO', 'NSE', 'MCX', 'CDS', 'BFO'], 'orderTypeList': ['MKT', 'L', 'SL'], 'productList': ['MIS', 'CNC', 'NRML', 'CO']}


In [4]:
token_df = pd.read_csv('https://developers.stocknote.com/doc/ScripMaster.csv')

In [5]:
desired_instruments = ['OPTCOM', 'FUTCOM']
token_df = token_df[token_df.instrument.isin(desired_instruments) & ~token_df.tradingSymbol.str.contains('TEST')].sort_values(['name', 'expiryDate'])
token_df

,exchange,exchangeSegment,symbolCode,tradingSymbol,name,lastPrice,instrument,lotSize,strikePrice,expiryDate,tickSize
132707,MFO,mcx_fo,259243_MFO,ALUMINI23DECFUT,ALUMINI,204.45,FUTCOM,1,0.0,2023-12-29,0.05
134114,MFO,mcx_fo,260595_MFO,ALUMINI24JANFUT,ALUMINI,208.35,FUTCOM,1,0.0,2024-01-31,0.05
134381,MFO,mcx_fo,260605_MFO,ALUMINI24FEBFUT,ALUMINI,210.00,FUTCOM,1,0.0,2024-02-29,0.05
133498,MFO,mcx_fo,426272_MFO,ALUMINI24MARFUT,ALUMINI,0.00,FUTCOM,1,0.0,2024-03-28,0.05
135792,MFO,mcx_fo,426301_MFO,ALUMINI24APRFUT,ALUMINI,0.00,FUTCOM,1,0.0,2024-04-30,0.05
...,...,...,...,...,...,...,...,...,...,...,...
136510,MFO,mcx_fo,259252_MFO,ZINCMINI23DECFUT,ZINCMINI,224.90,FUTCOM,1,0.0,2023-12-29,0.05
136594,MFO,mcx_fo,260609_MFO,ZINCMINI24JANFUT,ZINCMINI,228.95,FUTCOM,1,0.0,2024-01-31,0.05
136313,MFO,mcx_fo,260616_MFO,ZINCMINI24FEBFUT,ZINCMINI,230.50,FUTCOM,1,0.0,2024-02-29,0.05
136222,MFO,mcx_fo,426276_MFO,ZINCMINI24MARFUT,ZINCMINI,0.00,FUTCOM,1,0.0,2024-03-28,0.05


In [8]:
r = token_df.instrument.unique()
r

array(['FUTCOM', 'OPTCOM'], dtype=object)

In [6]:
# portfolio_nse = ['IDFCFIRSTB', 'LICHSGFIN', 
# 'GLENMARK', 'CIPLA', 'TATACOMM', 'ITC', 'ICICIGI', 'TATASTEEL',
# 'ADANIPORTS', 'BHARATFORG', 'GAIL', 'DIXON', 'TATAMOTORS',
# 'ASHOKLEY', 'TATAPOWER', 'NAUKRI', 'INDHOTEL', 'BHARTIARTL', 'NIFTY', 'BANKNIFTY',
# 'FINNIFTY']

portfolio_mcx = ['CRUDEOIL']

len(portfolio_mcx)


1

In [7]:
def update_dataframe(symbol, expiry, lotSize):
    oc_res = samco.get_option_chain(search_symbol_name=symbol,exchange=samco.EXCHANGE_MCX,expiry_date=expiry)
    oc_res = json.loads(oc_res)
    server_time = oc_res['serverTime']
    oc_df = pd.DataFrame(oc_res['optionChainDetails'])
    print("Option chain response==>", server_time)
    oc_df  = oc_df[['tradingSymbol', 'optionType', 'expiryDate', 'lastTradedPrice', 'openInterest', 'openInterestChange', 'volume', 'strikePrice']]
    oc_df = oc_df.astype({"strikePrice":float, "lastTradedPrice":float, "openInterest":int, "openInterestChange":int, "volume":int})
    oc_df = oc_df[oc_df.openInterest >0]
    oc_df = oc_df[oc_df.volume >0]
    ce_ocdf = oc_df[oc_df.optionType == 'CE']
    pe_ocdf = oc_df[oc_df.optionType == 'PE']
    option_chain = ce_ocdf.merge(pe_ocdf,on = 'strikePrice')
    option_chain = option_chain[['volume_x', 'openInterest_x', 'openInterestChange_x', 'lastTradedPrice_x', 'strikePrice', 'lastTradedPrice_y', 'openInterestChange_y','openInterest_y', 'volume_y']]  # Pass a list of column names inside double brackets
    # Obtain and rename columns
    new_columns = {
        'volume_x': 'Volume CE',
        'openInterest_x': 'OI',
        'openInterestChange_x': 'OI_CHANGE_CE',
        'lastTradedPrice_x': 'LTP_CE',
        'strikePrice': 'STRIKE',
        'lastTradedPrice_y': 'LTP_PE',
        'openInterestChange_y': 'OI_CHANGE_PE',
        'openInterest_y': 'OI',
        'volume_y': 'Volume PE',

    }
    oi_cols = ['volume_x', 'openInterest_x', 'openInterestChange_x', 'openInterestChange_y','openInterest_y', 'volume_y']
    option_chain[oi_cols] = (option_chain[oi_cols] / lotSize).astype(int)
    sorted_df = option_chain.sort_values(by='strikePrice', ascending=True)
    sorted_df = sorted_df.rename(columns=new_columns)
    sorted_df['Symbol'] = symbol
    sorted_df['Server_Time'] = server_time
    df = pd.DataFrame(sorted_df)
    
    return df

In [8]:
df_futures_mcx = token_df[token_df.instrument.isin(['FUTCOM', 'OPTCOM']) & 
token_df.name.isin(portfolio_mcx)]
df_futures_mcx = df_futures_mcx.groupby(['name']).head(1)
df_futures_mcx = df_futures_mcx.reset_index(drop=True)
df_futures_mcx
# df_fut.name.unique()

,exchange,exchangeSegment,symbolCode,tradingSymbol,name,lastPrice,instrument,lotSize,strikePrice,expiryDate,tickSize
0,MFO,mcx_fo,425434_MFO,CRUDEOIL24JAN5650PE,CRUDEOIL,66.9,OPTCOM,100,5650.0,2024-01-17,0.1


In [12]:
loop_duration = timedelta(minutes=25)

# Define the interval to wait between iterations (3 seconds)
iteration_interval = timedelta(seconds=6)

# Get the current time
start_time = datetime.now()


In [13]:


dfs = []
while datetime.now() - start_time < loop_duration:
    for index, row in df_futures_mcx.iterrows(): # make it better
        search_symbol_name = row['name']
        lot_size = row['lotSize']
        print(lot_size)
        # exchange = row['exchange']
        expiry_date = row['expiryDate']
        df  = update_dataframe(search_symbol_name, expiry_date, lot_size)
        # oc_res = samco.get_option_chain(search_symbol_name=search_symbol_name,exchange=samco.EXCHANGE_NFO,expiry_date=expiry_date)
        # oc_data = json.loads(oc_res)
        dfs.append(df)
        time.sleep(iteration_interval.total_seconds())

data_list = [df.to_dict() for df in dfs]
json_file_path = "data.json"

# Save the collected data to a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(data_list, json_file, indent=4)

print(f"Data collected and saved to {json_file_path}")



# df_OI = pd.concat(dfs, ignore_index=True)
# df_OI
# df_OI.rows

# oc_data

100


KeyError: 'optionChainDetails'

In [ ]:
# value=[{"symbol":"58129_NFO"}]
# samco.set_streaming_data(value)
# samco.start_streaming()
# json.loads(samco.get_quote(symbol_name='TATASTEEL23NOVFUT',exchange=samco.EXCHANGE_NFO))
# for i in df_futures_nse.items():

In [ ]:
df

,Volume CE,OI,OI_CHANGE_CE,LTP_CE,STRIKE,LTP_PE,OI_CHANGE_PE,OI,Volume PE,Symbol,Server_Time
23,0,1,1,403.7,6400.0,71.0,17,841,226,CRUDEOIL,30/10/23 23:27:58
22,2,354,1,482.0,6500.0,93.4,100,2539,459,CRUDEOIL,30/10/23 23:27:58
21,0,27,3,397.7,6600.0,124.5,26,907,242,CRUDEOIL,30/10/23 23:27:58
20,3,115,17,350.3,6700.0,159.3,-219,851,320,CRUDEOIL,30/10/23 23:27:58
19,0,2,-2,220.2,6750.0,186.8,26,38,7,CRUDEOIL,30/10/23 23:27:58
18,77,640,395,300.8,6800.0,202.9,-305,1612,569,CRUDEOIL,30/10/23 23:27:58
17,0,21,16,270.0,6850.0,226.8,18,74,16,CRUDEOIL,30/10/23 23:27:58
16,408,2720,1765,249.0,6900.0,251.9,-27,1654,671,CRUDEOIL,30/10/23 23:27:58
15,27,268,205,232.0,6950.0,270.4,-14,151,37,CRUDEOIL,30/10/23 23:27:58
14,1092,10798,4041,205.0,7000.0,303.0,-379,4349,984,CRUDEOIL,30/10/23 23:27:58


In [ ]:
import pandas as pd
json_file_path = "data.json"
data = pd.read_json(json_file_path, orient="records")

# You now have your data in the 'df' DataFrame
print(data)

                                             Volume CE  \
0    {'22': 2, '21': 0, '20': 2, '19': 0, '18': 63,...   
1    {'22': 2, '21': 0, '20': 2, '19': 0, '18': 63,...   
2    {'22': 2, '21': 0, '20': 2, '19': 0, '18': 63,...   
3    {'22': 2, '21': 0, '20': 2, '19': 0, '18': 63,...   
4    {'22': 2, '21': 0, '20': 2, '19': 0, '18': 63,...   
..                                                 ...   
234  {'23': 0, '22': 2, '21': 0, '20': 3, '19': 0, ...   
235  {'23': 0, '22': 2, '21': 0, '20': 3, '19': 0, ...   
236  {'23': 0, '22': 2, '21': 0, '20': 3, '19': 0, ...   
237  {'23': 0, '22': 2, '21': 0, '20': 3, '19': 0, ...   
238  {'23': 0, '22': 2, '21': 0, '20': 3, '19': 0, ...   

                                                    OI  \
0    {'22': 2821, '21': 1232, '20': 1129, '19': 38,...   
1    {'22': 2808, '21': 1208, '20': 1117, '19': 37,...   
2    {'22': 2786, '21': 1206, '20': 1116, '19': 37,...   
3    {'22': 2786, '21': 1206, '20': 1116, '19': 37,...   
4    {'22': 2

In [ ]:
json_objects = []
index_range = range(23)

for index in index_range:
    json_data = {}
    
    for key, value in data.items():
        if str(index) in value:
            json_data[key] = value[str(index)]

    json_objects.append(json_data)


# Convert the list of JSON objects to a DataFrame
json_df = pd.DataFrame(json_objects)

# Save the DataFrame to a JSON file
json_file_path = "data_crude.json"
json_df.to_json(json_file_path, orient="records", lines=True)

print(f"Data collected and saved to {json_file_path}")

Data collected and saved to data_crude.json
